<a href="https://colab.research.google.com/github/sheensta/retail_products_ensemble_deep_learning/blob/main/Code_for_all_image_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import(Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization)
from keras.preprocessing.image import ImageDataGenerator

#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_clean.csv')
#df['array'] = list(np.load('/content/drive/MyDrive/Colab Notebooks/np_img_array.npy'))

Model architecture 1

In [ ]:
X = list(df['array'])
le = LabelEncoder()
le.fit(df['categories'])
y = list(le.transform(df['categories']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
num_classes = 21
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Train our own model
model = Sequential() 
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", input_shape=(100, 100, 3))) 
model.add(BatchNormalization()) 
model.add(MaxPool2D((2, 2), strides=2, padding="same")) 
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu")) 
model.add(Dropout(0.2)) 
model.add(BatchNormalization()) 
model.add(MaxPool2D((2, 2), strides=2, padding="same")) 
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu")) 
model.add(BatchNormalization()) 
model.add(MaxPool2D((2, 2), strides=2, padding="same")) 
model.add(Flatten()) 
model.add(Dense(units=512, activation="relu")) 
model.add(Dropout(0.3)) 
model.add(Dense(units=num_classes, activation="softmax"))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False) 

augs_gen.fit(X_train)
history = model.fit(augs_gen.flow(X_train,y_train,batch_size=128),validation_data  = (X_test,y_test), steps_per_epoch  = 100,epochs = 15,verbose = 1)

Model architecture 2

In [ ]:
## Data Augmentation
from keras import layers
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomHeight(0.1),
        layers.experimental.preprocessing.RandomWidth(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)
    ]
)

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 1024]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=(100,100,3), num_classes=21)
model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 32,epochs = 15, verbose = 1)

Transfer learning VGG16

In [ ]:
#VGG16 transfer
from keras.applications.vgg16 import VGG16,preprocess_input

X = list(df['array'])
le = LabelEncoder()
le.fit(df['categories'])
y = list(le.transform(df['categories']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = preprocess_input(np.array(X_train))
X_test = preprocess_input(np.array(X_test))
num_classes = 21
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False) 

augs_gen.fit(X_train)

base_model = keras.applications.VGG16(include_top=False,
                  input_shape = (100,100,3),
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False


model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(21,activation='softmax'))

augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False) 

augs_gen.fit(X_train)
history = model.fit(augs_gen.flow(X_train,y_train,batch_size=128),validation_data  = (X_test,y_test), steps_per_epoch  = 100,epochs = 15,verbose = 1)

Transfer learning ResNet50

In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import ResNet50

X = list(df['array'])
le = LabelEncoder()
le.fit(df['categories'])
y = list(le.transform(df['categories']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = preprocess_input(np.array(X_train))
X_test = preprocess_input(np.array(X_test))
num_classes = 21
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

base_model = ResNet50(include_top=False,
                  input_shape = (100,100,3),
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(21,activation='softmax'))

augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False) 

augs_gen.fit(X_train)
history = model.fit(augs_gen.flow(X_train,y_train,batch_size=128),validation_data  = (X_test,y_test), steps_per_epoch  = 100,epochs = 15,verbose = 1)